Prepared by Aaron Schumacher
https://www.oreilly.com/learning/hello-tensorflow

In [1]:
import tensorflow as tf

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.scalar_summary(value.op.name, value)

summaries = tf.merge_all_summaries()

sess = tf.Session()
summary_writer = tf.train.SummaryWriter('log_simple_stats', sess.graph)

sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_writer.add_summary(sess.run(summaries), i)
    sess.run(train_step)

## Step by step

**Note:** Names have objects; objects don't have names.

In [1]:
foo = []
bar = foo
foo == bar
foo is bar # True, since the both point at the same list object

True

In [2]:
print id(foo), id(bar)

4568553664 4568553664


In [3]:
# Nesting lists is one way to represent a graph structure like a TensorFlow computation graph.
# Put a list inside itself, ~ a graph with one node, pointing to itself
foo.append(bar) 
foo

[[...]]

In [4]:
import tensorflow as tf

In [5]:
# An implicit default graph of interacting operations that run entirely outside Python.
graph = tf.get_default_graph() 
# Nodes of the TensorFlow graph are called “operations” or “ops”.
graph.get_operations() 

[]

In [6]:
input_value = tf.constant(1.0) # that constant now lives as a node, an operation, in the graph
input_value # a constant 32-bit float tensor of no dimension

<tf.Tensor 'Const:0' shape=() dtype=float32>

In [7]:
operations = graph.get_operations()
operations

In [8]:
operations[0].node_def # what's in TensorFlow's protocol buffer representation for the number one

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}

In [9]:
sess = tf.Session() # the session picks up the default graph by default
sess.run(input_value) # explicitly ask to evaluate input_value

1.0

In [10]:
# The simplest TensorFlow neuron with just one parameter, or weight.
# Often, even simple neurons also have a bias term and a non-identity activation function.
# The neuron's weight isn't going to be constant.
# We expect it to change in order to learn based on the “true” input and output used for training. 
# The weight will be a TensorFlow variable. 
# We'll give that variable a starting value of 0.8.
weight = tf.Variable(0.8)

In [11]:
for op in graph.get_operations(): print(op.name)

Const
Variable/initial_value
Variable
Variable/Assign
Variable/read


In [12]:
output_value = weight * input_value

In [13]:
for op in graph.get_operations(): print(op.name)

Const
Variable/initial_value
Variable
Variable/Assign
Variable/read
mul


In [14]:
op = graph.get_operations()[-1]
op.name

u'mul'

In [15]:
for op_input in op.inputs: print(op_input)

Tensor("Variable/read:0", shape=(), dtype=float32)
Tensor("Const:0", shape=(), dtype=float32)


In [16]:
init = tf.initialize_all_variables() # generates an operation which will initialize all variables
sess.run(init)

The result of tf.initialize_all_variables() will include initializers for all the variables currently in the graph, so if you add more variables you'll want to use tf.initialize_all_variables() again; a stale init wouldn't include the new variables.

In [17]:
sess.run(output_value) # 0.8 * 1.0 with 32-bit floats

0.80000001

In [18]:
# TensorBoard reads the name field that is stored inside each operation.
x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')

In [19]:
# The first argument when creating the SummaryWriter is an output directory name, 
# which will be created if it doesn't exist.
summary_writer = tf.train.SummaryWriter('log_simple_graph', sess.graph)

tensorboard --logdir=log_simple_graph

In [20]:
# We want the neuron to learn the function taking one to zero.
y_ = tf.constant(0.0)
# The goal is to minimize the loss
loss = (y - y_)**2

In [21]:
# Gradient descent optimizer so that we can update the weight based on the derivative of the loss. 
# The optimizer takes a learning rate to moderate the size of the updates.
optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)

In [22]:
grads_and_vars = optim.compute_gradients(loss)
sess.run(tf.initialize_all_variables())
sess.run(grads_and_vars[1][0]) # at this step = 0.8**2

1.6

In [20]:
# Let's apply the gradient, finishing the backpropagation.
# The optimizer subtracted the gradient times the learning rate, 1.6 * 0.025, 
# pushing the weight in the right direction.
sess.run(optim.apply_gradients(grads_and_vars))
sess.run(w)

0.75999999

In [21]:
# The training loop.
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)
for i in range(100):
    sess.run(train_step)

sess.run(y)

0.0044996012

In [22]:
# We want to follow what our system is predicting at every training step. 
sess.run(tf.initialize_all_variables())
for i in range(100):
    print('before step {}, y is {}'.format(i, sess.run(y)))
    sess.run(train_step)

before step 0, y is 0.800000011921
before step 1, y is 0.759999990463
before step 2, y is 0.722000002861
before step 3, y is 0.685899972916
before step 4, y is 0.651604950428
before step 5, y is 0.619024693966
before step 6, y is 0.588073432446
before step 7, y is 0.558669745922
before step 8, y is 0.530736267567
before step 9, y is 0.504199445248
before step 10, y is 0.478989481926
before step 11, y is 0.45504000783
before step 12, y is 0.432288020849
before step 13, y is 0.410673618317
before step 14, y is 0.390139937401
before step 15, y is 0.370632946491
before step 16, y is 0.352101296186
before step 17, y is 0.334496229887
before step 18, y is 0.317771404982
before step 19, y is 0.301882833242
before step 20, y is 0.286788702011
before step 21, y is 0.27244925499
before step 22, y is 0.25882679224
before step 23, y is 0.245885446668
before step 24, y is 0.233591169119
before step 25, y is 0.221911609173
before step 26, y is 0.210816025734
before step 27, y is 0.200275227427
befor

In [23]:
# The computation graph by adding operations that summarize its state. 
# Create an operation that reports the current value of y, the neuron's current output.
summary_y = tf.scalar_summary('output', y)

In [24]:
# Returns a string of protocol buffer text 
# that can be written to a log directory with a SummaryWriter.
summary_writer = tf.train.SummaryWriter('log_simple_stats')
sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_str = sess.run(summary_y)
    summary_writer.add_summary(summary_str, i)
    sess.run(train_step)